In [ ]:
#!/usr/bin/env python3
"""
Simple Ramulator Stats Extractor
Just run: python3 extract_ramulator_stats.py
"""

import re
import json
import csv
from pathlib import Path
from collections import defaultdict


def find_stats_file():
    """Find ramulator stats file in current directory"""
    # Look for common patterns
    patterns = ['*.txt', '*stats*', '*ramulator*']
    
    for pattern in patterns:
        files = list(Path('.').glob(pattern))
        if files:
            return files[0]
    
    # If nothing found, list all txt files
    txt_files = list(Path('.').glob('*.txt'))
    if txt_files:
        return txt_files[0]
    
    return None


def parse_stats_file(filename):
    """Parse the ramulator stats file"""
    metrics = {}
    channel_data = defaultdict(dict)
    current_metric = None
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            # Parse main metrics
            match = re.match(r'^\s*([a-zA-Z_.\d]+)\s+([\d.]+)\s+#\s*(.*)$', line)
            if match:
                metric_name = match.group(1)
                value = match.group(2)
                description = match.group(3)
                
                try:
                    value = float(value) if '.' in value else int(value)
                except:
                    pass
                
                metrics[metric_name] = {'value': value, 'description': description}
                current_metric = metric_name
            
            # Parse channel/indexed data
            elif current_metric:
                match = re.match(r'^\s*\[(.+?)\]\s+([\d.]+)', line)
                if match:
                    index = match.group(1)
                    value = match.group(2)
                    try:
                        value = float(value) if '.' in value else int(value)
                    except:
                        pass
                    channel_data[current_metric][index] = value
    
    return metrics, dict(channel_data)


def calculate_stats(metrics):
    """Calculate key statistics"""
    stats = {}
    
    # Bandwidth
    read_bw = metrics.get('ramulator.read_bandwidth', {}).get('value', 0)
    write_bw = metrics.get('ramulator.write_bandwidth', {}).get('value', 0)
    max_bw = metrics.get('ramulator.maximum_internal_bandwidth', {}).get('value', 1)
    
    stats['read_bandwidth_Bps'] = read_bw
    stats['write_bandwidth_Bps'] = write_bw
    stats['total_bandwidth_Bps'] = read_bw + write_bw
    stats['bandwidth_utilization_%'] = ((read_bw + write_bw) / max_bw * 100) if max_bw > 0 else 0
    
    # Latency
    stats['read_latency_avg_cycles'] = metrics.get('ramulator.read_latency_avg', {}).get('value', 0)
    
    # Row buffer
    hits = metrics.get('ramulator.row_hits', {}).get('value', 0)
    misses = metrics.get('ramulator.row_misses', {}).get('value', 0)
    conflicts = metrics.get('ramulator.row_conflicts', {}).get('value', 0)
    total = hits + misses + conflicts
    
    stats['row_hits'] = hits
    stats['row_misses'] = misses
    stats['row_conflicts'] = conflicts
    stats['row_hit_rate_%'] = (hits / total * 100) if total > 0 else 0
    stats['row_miss_rate_%'] = (misses / total * 100) if total > 0 else 0
    stats['row_conflict_rate_%'] = (conflicts / total * 100) if total > 0 else 0
    
    # Queue
    stats['avg_queue_length'] = metrics.get('ramulator.req_queue_length_avg', {}).get('value', 0)
    
    # Transactions
    stats['read_transaction_bytes'] = metrics.get('ramulator.read_transaction_bytes', {}).get('value', 0)
    stats['write_transaction_bytes'] = metrics.get('ramulator.write_transaction_bytes', {}).get('value', 0)
    stats['total_read_requests'] = metrics.get('ramulator.incoming_read_reqs_per_channel', {}).get('value', 0)
    
    return stats


def print_summary(stats):
    """Print nice summary"""
    print("\n" + "="*70)
    print("RAMULATOR STATISTICS SUMMARY".center(70))
    print("="*70)
    
    print("\n📊 BANDWIDTH")
    print(f"  Read:          {stats['read_bandwidth_Bps']:>15,} B/s ({stats['read_bandwidth_Bps']/1e9:.4f} GB/s)")
    print(f"  Write:         {stats['write_bandwidth_Bps']:>15,} B/s ({stats['write_bandwidth_Bps']/1e9:.4f} GB/s)")
    print(f"  Total:         {stats['total_bandwidth_Bps']:>15,} B/s ({stats['total_bandwidth_Bps']/1e9:.4f} GB/s)")
    print(f"  Utilization:   {stats['bandwidth_utilization_%']:>15.4f} %")
    
    print("\n⚡ LATENCY")
    print(f"  Avg Read:      {stats['read_latency_avg_cycles']:>15.2f} cycles")
    
    print("\n💾 ROW BUFFER")
    print(f"  Hits:          {stats['row_hits']:>15,} ({stats['row_hit_rate_%']:.2f}%)")
    print(f"  Misses:        {stats['row_misses']:>15,} ({stats['row_miss_rate_%']:.2f}%)")
    print(f"  Conflicts:     {stats['row_conflicts']:>15,} ({stats['row_conflict_rate_%']:.2f}%)")
    
    print("\n📈 TRANSACTIONS")
    print(f"  Read Bytes:    {stats['read_transaction_bytes']:>15,}")
    print(f"  Write Bytes:   {stats['write_transaction_bytes']:>15,}")
    print(f"  Read Requests: {stats['total_read_requests']:>15,}")
    
    print("\n📦 QUEUE")
    print(f"  Avg Length:    {stats['avg_queue_length']:>15.6f}")
    
    print("\n" + "="*70 + "\n")


def main():
    print("🔍 Looking for stats file...")
    
    # Find stats file
    stats_file = find_stats_file()
    if not stats_file:
        print("❌ No stats file found in current directory!")
        print("   Place your stats.txt file here and run again.")
        return
    
    print(f"✓ Found: {stats_file}")
    
    # Parse
    print(f"📖 Parsing {stats_file}...")
    metrics, channel_data = parse_stats_file(stats_file)
    print(f"✓ Parsed {len(metrics)} metrics")
    
    # Calculate stats
    stats = calculate_stats(metrics)
    
    # Print summary
    print_summary(stats)
    
    # Save outputs
    base_name = stats_file.stem
    
    # Save JSON
    json_file = f"{base_name}_summary.json"
    output_data = {
        'summary_stats': stats,
        'all_metrics': {k: v['value'] for k, v in metrics.items()},
        'channel_distribution': channel_data
    }
    with open(json_file, 'w') as f:
        json.dump(output_data, f, indent=2)
    print(f"💾 Saved: {json_file}")
    
    # Save CSV (key metrics only)
    csv_file = f"{base_name}_summary.csv"
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Metric', 'Value'])
        for key, value in stats.items():
            writer.writerow([key, value])
    print(f"💾 Saved: {csv_file}")
    
    # Save detailed CSV
    csv_detailed = f"{base_name}_detailed.csv"
    with open(csv_detailed, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Metric', 'Value', 'Description'])
        for metric_name, data in sorted(metrics.items()):
            writer.writerow([metric_name, data['value'], data.get('description', '')])
    print(f"💾 Saved: {csv_detailed}")
    
    print("\n✅ Done! Check the generated files.")


if __name__ == '__main__':
    main()